In [ ]:
!nvidia-smi

In [ ]:
pip install uv

In [ ]:
!uv pip install torch

In [ ]:
pip freeze

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [10]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:

# Transform and full dataset (train + test)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Use only train set for training
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [12]:
# Model
class DigitCNN(nn.Module):
    def __init__(self):
        super(DigitCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # 28 → 14
        x = self.pool(self.relu(self.conv2(x)))  # 14 → 7
        x = x.view(-1, 32 * 7 * 7)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

In [13]:
model = DigitCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Train loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/5, Loss: 0.1972
Epoch 2/5, Loss: 0.0538
Epoch 3/5, Loss: 0.0358
Epoch 4/5, Loss: 0.0285
Epoch 5/5, Loss: 0.0205


In [15]:
# Save model
os.makedirs('saved_model', exist_ok=True)
torch.save(model.state_dict(), 'saved_model/mnist_cnn.pth')
print("✅ Model saved to 'saved_model/mnist_cnn.pth'")

✅ Model saved to 'saved_model/mnist_cnn.pth'
